# Imports

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from importlib import reload
import adaptive_confound.utils as acu
import adaptive_confound.control as acc
import adaptive_confound.topic_model as actm

/home/virgile/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/virgile/.conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [61]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sp
import pickle
import os.path
import tensorflow as tf
import itertools as it

from datetime import datetime
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.metrics import f1_score, precision_score, recall_score
from copy import deepcopy
from scipy.stats import mode
from collections import Counter, defaultdict

pd.options.display.max_colwidth = 200
pd.options.display.max_columns = 100

In [51]:
from keras import backend as K
from keras.layers import Dense, Input, Lambda, Concatenate, Dropout
from keras.models import Model, Sequential
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, Callback
from keras.datasets import reuters
from keras.losses import binary_crossentropy, mse
import keras.utils.vis_utils as kvu

In [4]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
sess = tf.Session(config=config)
set_session(sess)

In [5]:
ystr = 'location'
zstr = 'gender'
twitter = acu.read_pickle("/data/virgile/confound/adaptive/in/twitter_dataset_y={}_z={}.pkl".format(ystr, zstr))

In [6]:
def shuffle_dataset(d, seed=1234):
    idx = np.arange(d.get_size())
    np.random.seed(seed)
    np.random.shuffle(idx)
    d.X = d.X[idx]
    d.y = d.y[idx]
    d.z = d.z[idx]

In [7]:
shuffle_dataset(twitter)

In [8]:
twitter

< Dataset: size=6000, p(y)=0.50, p(z)=0.50, bias=0.50, parent=None >

In [38]:
twitter.create_biased_datasets(1000, np.linspace(.1,.9,9), k=1)

In [153]:
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical

class Metrics(Callback):
    def on_epoch_end(self, batch, logs={}):
        y_pred = self.model.predict(self.validation_data[0])[0].argmax(axis=1)
        y_true = self.validation_data[1].argmax(axis=1)
        z_pred = self.model.predict(self.validation_data[0])[1].argmax(axis=1)
        z_true = self.validation_data[2].argmax(axis=1)
        
        self.f1_y = f1_score(y_true, y_pred)
        self.f1_z = f1_score(z_true, z_pred)
        
        print()
        print("val_f1_y: {:.4f}".format(self.f1_y))
        print("val_f1_z: {:.4f}".format(self.f1_z), "z_pred: #1: {:d} / count: {:d}".format(z_pred.sum(), z_pred.size))
        
        return
metrics = Metrics()

## Deep Residual

In [109]:
class DR_BOW:
    def __init__(self, x_dim, c_dim, xl=100, cl=20, pl=20):
        self.x_dim = x_dim
        self.c_dim = c_dim
        self.xl = xl
        self.cl = cl
        self.pl = pl
        
        self._build_model()

    def _build_model(self):
        # Model predicting Y from C
        c_input = Input(shape=(self.c_dim,), name="c_input")
        l = Dense(self.cl, activation="relu")(c_input)
        y_hat_prime = Dense(2, activation="softmax", name="y_hat_prime")(l)
        c2y = Model(c_input, y_hat_prime)

        # Reduce dimension of text input
        x_input = Input(shape=(self.x_dim,), name="x_input")
        e = Dense(self.xl, activation="relu", name="e")(x_input)
        
        # Concatenate latent representation of text input and prediction of y | c
        l = Concatenate()([e, y_hat_prime])
        l = Dense(self.pl, activation="relu")(l)
        y_hat = Dense(2, activation="softmax", name="y_hat")(l)
        self.model = Model([x_input, c_input], [y_hat_prime, y_hat])
        self.model.compile(optimizer="adam", loss="categorical_crossentropy")
        
    def fit(self, d, *args, **kwargs):
        tocat = lambda x: to_categorical(x, num_classes=2)
        z = d.z.reshape(-1,1)
        return self.model.fit([d.X, z], [tocat(d.y), tocat(d.y)], *args, **kwargs)
    
    def predict(self, d, *args, **kwargs):
        return self.model.predict([d.X, d.z])[1].argmax(axis=1)

In [110]:
m = DR_BOW(twitter.features.size, 1)
kvu.plot_model(m.model, 'DRBOW.png')

In [112]:
m.fit(twitter, epochs=100, validation_split=.5)

Train on 3000 samples, validate on 3000 samples
Epoch 1/100
3000/3000 [==============================] - 2s 550us/step - loss: 0.7037 - y_hat_prime_loss: 0.6927 - y_hat_loss: 0.0110 - val_loss: 1.3247 - val_y_hat_prime_loss: 0.6954 - val_y_hat_loss: 0.6293
Epoch 2/100
3000/3000 [==============================] - 2s 560us/step - loss: 0.7034 - y_hat_prime_loss: 0.6927 - y_hat_loss: 0.0107 - val_loss: 1.3404 - val_y_hat_prime_loss: 0.6948 - val_y_hat_loss: 0.6456
Epoch 3/100
3000/3000 [==============================] - 2s 588us/step - loss: 0.7032 - y_hat_prime_loss: 0.6926 - y_hat_loss: 0.0106 - val_loss: 1.3577 - val_y_hat_prime_loss: 0.6950 - val_y_hat_loss: 0.6627
Epoch 4/100
3000/3000 [==============================] - 1s 499us/step - loss: 0.7031 - y_hat_prime_loss: 0.6926 - y_hat_loss: 0.0105 - val_loss: 1.3769 - val_y_hat_prime_loss: 0.6948 - val_y_hat_loss: 0.6821
Epoch 5/100
3000/3000 [==============================] - 2s 544us/step - loss: 0.7031 - y_hat_prime_loss: 0.6927 - y

## Adversarial

In [171]:
# Reverse gradient layer from https://github.com/michetonu/gradient_reversal_keras_tf/blob/master/flipGradientTF.py
# Added compute_output_shape for Keras 2 compatibility
from keras.engine import Layer
import keras.backend as K

def reverse_gradient(X, hp_lambda):
    '''Flips the sign of the incoming gradient during training.'''
    try:
        reverse_gradient.num_calls += 1
    except AttributeError:
        reverse_gradient.num_calls = 1

    grad_name = "GradientReversal%d" % reverse_gradient.num_calls

    @tf.RegisterGradient(grad_name)
    def _flip_gradients(op, grad):
        return [tf.negative(grad) * hp_lambda]

    g = K.get_session().graph
    with g.gradient_override_map({'Identity': grad_name}):
        y = tf.identity(X)

    return y

class GradientReversal(Layer):
    '''Flip the sign of gradient during training.'''
    def __init__(self, hp_lambda, **kwargs):
        super(GradientReversal, self).__init__(**kwargs)
        self.supports_masking = False
        self.hp_lambda = hp_lambda

    def build(self, input_shape):
        self.trainable_weights = []

    def call(self, x, mask=None):
        return reverse_gradient(x, self.hp_lambda)

    def get_output_shape_for(self, input_shape):
        return input_shape
    
    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = {'hp_lambda': self.hp_lambda}
        base_config = super(GradientReversal, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [186]:
class A_BOW:
    def __init__(self, x_dim, xl=100, tl=50, cl=50, inv_factor=1):
        self.x_dim = x_dim
        self.xl = xl
        self.tl = tl
        self.cl = cl
        self.inv_factor = inv_factor
        self._build_model()
    
    def _build_model(self):
        x_input = Input((self.x_dim,), name="x_input")
        e = Dense(self.xl, activation="relu", name="e")(x_input)
    
        l = Dense(self.tl, activation="relu")(e)
        t = Dense(2, activation="softmax", name="y")(l)
        
        l = GradientReversal(self.inv_factor)(e)
        l = Dense(self.cl, activation="relu")(l)
        c = Dense(2, activation="softmax", name="z")(l)
        
        self.model = Model(x_input, [t, c])
        self.model.compile(optimizer="adam",
                           loss=["categorical_crossentropy", "categorical_crossentropy"],)
#                            loss_weights=[1, -1 * self.inv_factor])
        
    def fit(self, d, *args, **kwargs):
        tocat = lambda x: to_categorical(x, num_classes=2)
        vd = kwargs.get("validation_data", ())
        if type(vd) != tuple:
            kwargs["validation_data"] = (vd.X, [tocat(vd.y), tocat(vd.z)])
        cbs = kwargs.get("callbacks", [])
        if "Metrics" not in [cb.__class__.__qualname__ for cb in cbs]:
            cbs.append(metrics)
            kwargs["callbacks"] = cbs
        return self.model.fit(d.X, [tocat(d.y), tocat(d.z)], *args, **kwargs)
    
    def predict(self, d, *args, **kwargs):
        return self.model.predict(d.X)[0]

In [191]:
m = A_BOW(twitter.features.size, xl=50, tl=10, cl=10,
          inv_factor=.5)
kvu.plot_model(m.model, 'ABOW.png')

In [192]:
m.fit(twitter.biased_datasets[0], epochs=100, batch_size=10, validation_data=twitter.biased_datasets[-1])

Train on 1000 samples, validate on 1000 samples
Epoch 1/100
 950/1000 [===========================>..] - ETA: 0s - loss: 7.3953 - y_loss: 0.5260 - z_loss: 6.8692
val_f1_y: 0.6551
val_f1_z: 0.0268 z_pred: #1: 14 / count: 1000
1000/1000 [==============================] - 4s 4ms/step - loss: 7.3505 - y_loss: 0.5178 - z_loss: 6.8327 - val_loss: 6.5324 - val_y_loss: 0.9879 - val_z_loss: 5.5445
Epoch 2/100
 930/1000 [==========================>...] - ETA: 0s - loss: 5.0178 - y_loss: 0.1884 - z_loss: 4.8295
val_f1_y: 0.1491
val_f1_z: 0.6746 z_pred: #1: 1000 / count: 1000
1000/1000 [==============================] - 2s 2ms/step - loss: 5.2310 - y_loss: 0.1984 - z_loss: 5.0327 - val_loss: 10.0568 - val_y_loss: 3.0222 - val_z_loss: 7.0346
Epoch 3/100
 930/1000 [==========================>...] - ETA: 0s - loss: 4.3636 - y_loss: 0.1277 - z_loss: 4.2359
val_f1_y: 0.6908
val_f1_z: 0.3976 z_pred: #1: 638 / count: 1000
1000/1000 [==============================] - 2s 2ms/step - loss: 4.4441 - y_loss: 0